In [127]:
import pandas as pd
import sqlite3 as sq
import numpy as np


Just imported libraries

In [128]:
stud_name = np.array([], dtype=object)
stud_age = np.array([], dtype=int)
stud_score = np.array([], dtype=int)
stud_passed = np.array([], dtype=bool)
# New features for ML
stud_attendance = np.array([], dtype=float)
stud_hw_avg = np.array([], dtype=float)
stud_projects = np.array([], dtype=int)

In [130]:
data = pd.DataFrame({
    'Name': stud_name,
    'Age': stud_age,
    'Score': stud_score,
    'Passed': stud_passed,
    'Attendance': stud_attendance,
    'HW_Avg': stud_hw_avg,
    'Projects': stud_projects
}, index=range(len(stud_name)), dtype=object)

In [132]:
def add_student():
    global stud_name, stud_age, stud_score, stud_passed
    global stud_attendance, stud_hw_avg, stud_projects
    stud_name = np.append(stud_name, input('Enter name: '))
    stud_age = np.append(stud_age, input('Enter age: '))
    x = int(input('Enter score: '))
    stud_score = np.append(stud_score, x)
    if x >= 60:
        stud_passed = np.append(stud_passed, True)
    else:
        stud_passed = np.append(stud_passed, False)
            # New features input
    att = float(input('Enter attendance (%): '))
    hw = float(input('Enter homework average: '))
    proj = int(input('Enter number of projects: '))
    stud_attendance = np.append(stud_attendance, att)
    stud_hw_avg = np.append(stud_hw_avg, hw)
    stud_projects = np.append(stud_projects, proj)
    with sq.connect('students.db') as con:
        df = pd.DataFrame({'name': stud_name,
                           'age': stud_age,
                           'score': stud_score,
                           'passed': stud_passed,
                           'stud_att': stud_attendance,
                           'H/W AVG' :stud_hw_avg ,
                           'Projects': stud_projects},
                            dtype=object)
        df.to_sql('students', con, if_exists='replace', index=False)

Function to add a student to the database

In [134]:
def show_students():
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)
        df['passed'] = df['passed'].astype(bool)
        display(df)

Function to show all students

In [135]:
def clear_database():
    global stud_name, stud_age, stud_score, stud_passed
    stud_name = np.array([], dtype=object)
    stud_age = np.array([], dtype=int)
    stud_score = np.array([], dtype=int)
    stud_passed = np.array([], dtype=bool)
    with sq.connect('students.db') as con:
        con.execute('DELETE FROM students')
        con.commit()
    print("All data has been cleared!")

    Function to clear the database

In [136]:
def score_mean():
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)
        mean_score = df['score'].mean()
        print(f"Mean score: {mean_score:.2f}")

In [137]:
def max_score():
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)
        max_score = df['score'].max()
        print(f"Max score: {max_score}")

In [138]:
def min_score():
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)
        min_score = df['score'].min()
        print(f"Min score: {min_score}")

In [141]:
import numpy as np
import pandas as pd
import sqlite3 as sq

def predict_values_sgd(L=20.0, lr=0.001, epochs=1000):
    global w
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)

    if len(df) < 2:
        print('Not enough data!')
        return None, None

    X = np.c_[
        np.ones(len(df)),
        df['age'].astype(float).values,
        df['stud_att'].astype(float).values,
        df['H/W AVG'].astype(float).values,
        df['Projects'].astype(float).values
    ]
    Y = df['score'].astype(float).values.reshape(-1, 1)

    n_samples, n_features = X.shape
    w = np.zeros((n_features, 1))

    for epoch in range(epochs):

        indices = np.random.permutation(n_samples)

        for i in indices:
            xi = X[i].reshape(1, -1)
            yi = Y[i]


            y_pred = xi @ w
            error = y_pred - yi

            # градиент
            grad = xi.T @ error + L * w
            grad[0] -= L * w[0]


            w -= lr * grad

        if epoch % 100 == 0:
            loss = np.mean((X @ w - Y) ** 2)
            print(f"Epoch {epoch}: loss = {loss:.4f}")

    print("Model trained with stochastic gradient descent")
    return w, df

In [147]:
try:
    while True:
        choice = input("""Enter your choice: (1/2/3/4/5)

                            1. Add student
                            2. Show studentsв
                            3. Clear database
                            4. Calculate mean score
                            5. Calculate min score
                            6. Calculate max score
                            7. Predict Score
                            8. Exit
                            """).strip()
        match choice:
            case '1':
                add_student()
            case '2':
                show_students()
                break
            case '3':
                clear_database()
                break
            case '4':
                score_mean()
                break
            case '5':
                min_score()
                break
            case '6':
                max_score()
                break
            case '7':
                predict_stud()
            case '8':
                break
            case _:
                print("Invalid choice. Please try again.")

except ValueError:
    print("Invalid choice. Please try again.")
except KeyboardInterrupt:
    print(" ")
except Exception as e:
    print(f"An error occurred: {e}")

,name,age,score,passed,stud_att,H/W AVG,Projects
0,Max,20,70,True,70.0,70.0,35
1,Misterr,11,60,True,60.0,60.0,30


In [144]:
def predict_stud():
    global Stud_Age , Stud_at , Stud_HW , StudProj
    Stud_Age= int(input('Age:  '))
    Stud_at = float(input('Attendance of Student: '))
    Stud_HW = float(input('H/W Percent: '))
    StudProj = int(input('How many projects students does ?'))



    x_new = np.array([1, Stud_Age , Stud_at, Stud_HW, StudProj])
    y_pred = np.dot(w, x_new)

    if y_pred > 100:
        y_pred = 100

    print(f"Score: {y_pred:.2f}")

In [145]:
import random

def generate_students(n=50):
    global stud_name, stud_age, stud_score, stud_passed
    global stud_attendance, stud_hw_avg, stud_projects

    names = [f"Student_{i}" for i in range(1, n+1)]
    ages = np.random.randint(15, 25, size=n)               # возраст 15–24
    scores = np.random.randint(30, 100, size=n)            # баллы 30–99
    passed = scores >= 60                                  # кто прошёл
    attendance = np.random.uniform(50, 100, size=n)        # посещаемость %
    hw_avg = np.random.uniform(40, 100, size=n)            # средняя оценка за дз
    projects = np.random.randint(0, 10, size=n)            # количество проектов

    stud_name = np.append(stud_name, names)
    stud_age = np.append(stud_age, ages)
    stud_score = np.append(stud_score, scores)
    stud_passed = np.append(stud_passed, passed)
    stud_attendance = np.append(stud_attendance, attendance)
    stud_hw_avg = np.append(stud_hw_avg, hw_avg)
    stud_projects = np.append(stud_projects, projects)

    with sq.connect('students.db') as con:
        df = pd.DataFrame({
            'name': stud_name,
            'age': stud_age,
            'score': stud_score,
            'passed': stud_passed,
            'attendance': stud_attendance,
            'hw_avg': stud_hw_avg,
            'projects': stud_projects
        })
        df.to_sql('students', con, if_exists='replace', index=False)

    print(f"{n} students added to database!")



    generate_students(100)  # например 100 студентов